# Aula #26 – Processamento de Linguagem Natural & Análise de Sentimento

# Word2vec

Já vimos antes que é possível transformar um texto em _features_ numéricas. Uma sofisticação do método _Bag of words_ é incorporar o contexto das palavras vizinhas nessas _features_ (é comum chamar o vetor de _features_ numéricas de _embedding_).

Imagine que nossa janela de contexto (context window) tem tamanho 5 (2 palavras _antes_ e 2 palavras _depois_ da palavra _central_).

Então, se a frase fosse `The quick brown fox jumps over the lazy dog`, teríamos as seguintes janelas:

<img src="data/nb_figs/windows_word2vec.png" width="600"/>

Para cada uma das janelas formadas, temos o vetor correspondente a elas (usando o _Bag of words_ binário - com apenas 0s e 1s; também chamado de `one-hot encoding`):

<img src="data/nb_figs/one_hot_encoding_word2vec.png" width="600"/>

Há duas arquiteturas possíveis para se obter os `embeddings` word2vec. Uma delas é chamada de `CBoW` (_Continuous Bag of Words_) e outra é chamada de `Skip gram`. Aqui, vamos focar no `Skip gram`, que considera como input o vetor da palavra central da janela, e como output, os vetores do contexto. O objetivo do algoritmo é aprender os pesos da _hidden layer_, de forma que as probabilidades finais sejam condizentes com as co-ocorrências das palavras em nosso _corpus_ de documentos.

<img src="data/nb_figs/nn_word2vec_large.png" width="800"/>

Ao final do treinamento, a matriz correspondente à _hidden layer_, com 10 mil (tamanho do vocabulário) linhas e 300 (quantidade de dimensões do _embedding_) colunas será tal que cada linha representará o embedding de uma palavra do vocabulário.

Para saber mais sobre `word2vec`, leia em:

* http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
* https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/
* https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

## Similaridade entre ingredientes - uma aplicação do _word2vec_ a um dataset de receitas

O dataset utilizado é um dos datasets do site [Recipe box](https://eightportions.com/datasets/Recipes).

A ideia é treinar um modelo `word2vec` usando a biblioteca [gensim](https://radimrehurek.com/gensim/index.html) e depois construirmos uma aplicação pela qual seja possível obter uma lista dos ingredientes mais similares a um determinado ingrediente. Vamos tentar?

### Leitura do dataset

In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
with open('data/datasets/recipes/recipes_raw_nosource_ar.json') as f:
    recipes_list = list(json.load(f).values())

In [ ]:
df = pd.DataFrame(recipes_list)

In [ ]:
df = df[df['instructions'].str.len() > 0]

In [ ]:
df.head()

In [ ]:
len(df)

### Normalização da coluna `instructions`

In [ ]:
import string
import nltk
import re

In [ ]:
translation_table = str.maketrans({key: ' ' for key in string.punctuation}) 
def remove_punctuation(text):
    return text.translate(translation_table)

In [ ]:
digits_regex = re.compile(r'[0-9]')
def remove_digits(text):
    return digits_regex.sub('', text)

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(text):
    return [word for word in text.split() if word not in stopwords]

In [ ]:
def text_to_normalized_tokens(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = remove_digits(text)
    text = remove_stopwords(text)
    return text

In [ ]:
df['norm_instructions'] = df['instructions'].apply(text_to_normalized_tokens)

In [ ]:
df = df[df['norm_instructions'].str.join('').str.len() > 0]

In [ ]:
df.head()

## Treinamento do _word2vec_

**Tarefa:** Treine um modelo word2vec usando os dados da coluna `ingredients` (`words_list`)

1. crie uma variável chamada `word_list` que é uma lista com os ingredientes

2. defina as variáveis `size` (tamanho do embedding) e `window` (tamanho da janela que deslizará pelas listas de palavras

3. faça o treinamento do word2vec, passando como parâmetros `word_list`, `size=size`, `window=window`, `min_count=1` e `workers=4` (vc pode aumentar esse número ou diminuir de acordo com a quantidade de cpus que vc tiver disponível)

Dica: Leia a documentação sobre a classe `Word2Vec`

<!-- 
words_list = df['norm_instruction'].tolist()
size = 300
window = 5
model = Word2Vec(words_list, size=size, window=window, min_count=1, workers=4)
-->

In [ ]:
from gensim.models import Word2Vec

In [ ]:
?Word2Vec

In [ ]:
# 1
words_list = ###

# 2
size = ###
window = ###

# 3
model = Word2Vec(###

Se você quiser salvar o modelo, você precisa apenas utilizar o método `save`, passando como parâmetro o local em que deseja que o modelo seja salvo.

Por exemplo:
```python
model.save('data/gensim.model')
```

### Similaridade entre vetores

Em modelos vetoriais de linguagem, em geral, utiliza-se a similaridade de cosseno como medida de similaridade entre dois vetores, já que ela captura a noção de que vetores apontando para a mesma direção são próximos.

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
lime_vec = model.wv['lime']
lemon_vec = model.wv['lemon']

In [ ]:
def similarity_between_vec(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [ ]:
similarity_between_vec(lime_vec, lemon_vec)

### Termos mais comuns

Vamos ver quais são os termos mais comuns do dataset?

In [ ]:
from collections import Counter

In [ ]:
from itertools import chain

In [ ]:
all_words = chain.from_iterable(df['norm_instructions'].tolist())

In [ ]:
Counter(all_words).most_common(20)

### Os mais próximos

Um método legal do objeto `Word2VecKeyedVectors` é o `most_similar`, que retorna as palavras mais similares a uma determinada palavra. Note que podemos modificar a quantidade de itens retornados, colocando um valor para parâmetro `topn` (por padrão, ele é 10).

**Tarefa:** brinque até ficar satisfeito.

As relações fazem sentido?

In [ ]:
?model.wv.most_similar

In [ ]:
model.wv.most_similar('rice')

## Visualização das relações entre os ingredientes

Vamos agora construir funções que permitem:

1. buscar o nome de um ingrediente
2. retornar os termos mais próximos (que não são ele mesmo)

In [ ]:
VOCAB = set(model.wv.vocab.keys())

**Tarefa:** Complete a função abaixo, que dado um termo (`word`), retorna os `n` termos mais similares. Além disso, se a palavra não está no vocabulário, imprime uma mensagem que avisa o usuário que a palavra não existe no vocabulário.

Lembre-se de passar o parâmetro `topn` (`topn=n`) para o método `model.wv.most_similar` para retornar os `n` termos.

<!-- 
def get_similar(word, n=10):
    if word not in VOCAB:
        print(f'A palavra "{word}" não está em nosso vocabulário!')
        exit(1)
    
    most_similar_words = []
    for similar_word, distance in model.wv.most_similar(word, topn=n+1):
        most_similar_words.append(similar_word)
        
    return most_similar_words
-->

In [ ]:
def get_similar(word, n=10):
    if word not in VOCAB:
        print(###
        exit(1)
    
    most_similar_words = []
    for similar_word, distance in model.wv.most_similar(###):
        most_similar_words.append(###
        
    return most_similar_words

In [ ]:
def search_text(text, max_display=15):
    words_with_text = [word for word in VOCAB if text.lower() in word][:max_display]
    if text in words_with_text:
        words_with_text = [text] + [word for word in words_with_text if text != word]
    return words_with_text

In [ ]:
search_text('rosemary')

In [ ]:
get_similar('rosemary')

**Tarefa bônus:** descubra a similaridade entre uma receita e outra receita através dos embeddings que treinamos acima. Isso envolve utilizar um embedding para a instrução (a nossa "sentença", que é um conjunto de palavras).

Veja exemplos da criação de embeddings para sentença a partir dos embeddings da palavra [neste post](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/).

Após a criação dos embeddings das receitas, basta comparará-las!